In [1]:
import polars as pl
import polars_ds as pds
import numpy as np

# This notebook illustrates the basic usage of this package

You need to create an environment with this package installed to run this notebook. (usually latest version)

# Num Extensions

In [2]:
size = 10_000
df = pl.DataFrame({
    "f": np.sin(list(range(size)))
    , "time_idx": range(size)
    , "dummy": ["a"] * (size // 2) + ["b"] * (size // 2)
    , "a": np.random.random(size = size)
    , "b": np.random.random(size = size)
    , "x1" : range(size)
    , "x2" : range(size, size + size)
    , "y": range(-size, 0)
    , "actual": np.round(np.random.random(size=size)).astype(np.int32)
    , "predicted": np.random.random(size=size)
    , "dummy_groups":["a"] * (size//2) + ["b"] * (size//2) 
})
df.head()

f,time_idx,dummy,a,b,x1,x2,y,actual,predicted,dummy_groups
f64,i64,str,f64,f64,i64,i64,i64,i32,f64,str
0.0,0,"""a""",0.320392,0.680298,0,10000,-10000,1,0.205471,"""a"""
0.841471,1,"""a""",0.552808,0.600066,1,10001,-9999,1,0.10919,"""a"""
0.909297,2,"""a""",0.994927,0.644871,2,10002,-9998,1,0.50784,"""a"""
0.14112,3,"""a""",0.461395,0.376017,3,10003,-9997,0,0.289778,"""a"""
-0.756802,4,"""a""",0.891036,0.29814,4,10004,-9996,1,0.431257,"""a"""


In [3]:
# Column-wise Jaccard Similarity. Result should be 0 as they are distinct
df.select(
    pds.query_jaccard_col("x1", pl.col("x2"))
)

x1
f64
0.0


In [4]:
# FFT. First is real part, second is complex part
# By default, this behaves the same as np's rfft, which returns a non-redundant 
# compact representation of fft output.
df.select(
    pds.rfft("f")
).head()

f
"array[f64, 2]"
"[1.939505, 0.0]"
"[1.939506, 0.000209]"
"[1.939508, 0.000418]"
"[1.939512, 0.000627]"
"[1.939518, 0.000835]"


In [5]:
# FFT. But return the full length
df.select(
    pds.rfft("f", return_full=True)
).shape

(10000, 1)

In [6]:
# Convolution (by FFT). 
# Modes: `same`, `left` (left-aligned same), `right` (right-aligned same), `valid` or `full`
# Currently slower than SciPy but provides parallelism because of Polars
df.select(
    pds.convolve("f", [-1, 0, 0, 0, 1], mode = "full"),
    pds.convolve("a", [-1, 0, 0, 0, 1], mode = "full"),
    pds.convolve("b", [-1, 0, 0, 0, 1], mode = "full"),
).head()

f,a,b
f64,f64,f64
0.0,-0.320392,-0.680298
-0.841471,-0.552808,-0.600066
-0.909297,-0.994927,-0.644871
-0.14112,-0.461395,-0.376017
0.756802,-0.570644,0.382158


In [7]:
# Least Square (Linear Regression)
df.select(
    pds.query_lstsq(
        pl.col("x1"), pl.col("x2"),
        target = pl.col("y"),
        add_bias=False
    )
)

y
list[f64]
"[2.0, -1.0]"


In [8]:
df.select(
    pds.query_lstsq_report(
        # str | pl.Expr
        "x1", "x2",
        target = pl.col("y"),
        add_bias=False
    ).alias("report")
).unnest("report")

idx,coeff,std_err,t,p>|t|
u16,f64,f64,f64,f64
0,2.0,8.5322e-17,2.3441e16,0.0
1,-1.0,3.2247e-17,-3.1010e16,0.0


In [9]:
df.lazy().select(
    pds.query_lstsq(
        pl.col("x1"), pl.col("x2"),
        target = "y", # We can either put pl.col("y") here or just the string "y"
        add_bias=False
    )
).collect()

y
list[f64]
"[2.0, -1.0]"


In [10]:
df.select(
    "dummy",
    pds.query_lstsq(
        pl.col("x1"), pl.col("x2"),
        target = pl.col("y"),
        add_bias=False
    ).over(pl.col("dummy"))
).head() 

dummy,coeffs
str,list[f64]
"""a""","[2.0, -1.0]"
"""a""","[2.0, -1.0]"
"""a""","[2.0, -1.0]"
"""a""","[2.0, -1.0]"
"""a""","[2.0, -1.0]"


In [11]:
# If you want prediction and residue instead of coefficients
df.select(
    "x1",
    "x2",
    "y",
    pds.query_lstsq(
        "x1", pl.col("x2"),
        target = "y",
        add_bias=False, 
        return_pred=True
    ).alias("prediction")
).unnest("prediction").head()

x1,x2,y,pred,resid
i64,i64,i64,f64,f64
0,10000,-10000,-10000.0,-1.8190e-12
1,10001,-9999,-9999.0,-1.8190e-12
2,10002,-9998,-9998.0,-1.8190e-12
3,10003,-9997,-9997.0,-1.8190e-12
4,10004,-9996,-9996.0,-1.8190e-12


In [12]:
df.group_by("dummy").agg(
    pds.query_lstsq(
        pl.col("x1"), pl.col("x2"),
        target = pl.col("y"),
        add_bias=False
    )
)

dummy,coeffs
str,list[f64]
"""b""","[2.0, -1.0]"
"""a""","[2.0, -1.0]"


In [13]:
# Rolling regression, kind of slow rn
df.lazy().rolling(
    index_column = pl.col("time_idx").set_sorted(),
    period = "30i",
    # offset = "-1i"
).agg(
    pds.query_lstsq(pl.col("x1"), pl.col("x2"), target = pl.col("y"), add_bias=False).alias("coefficients")
).slice(offset = 30).select(
    "time_idx",
    "coefficients",
).collect().head(10)

time_idx,coefficients
i64,list[f64]
30,"[2.0, -1.0]"
31,"[2.0, -1.0]"
32,"[2.0, -1.0]"
33,"[2.0, -1.0]"
34,"[2.0, -1.0]"
35,"[2.0, -1.0]"
36,"[2.0, -1.0]"
37,"[2.0, -1.0]"
38,"[2.0, -1.0]"


In [14]:
# Conditional Entropy, should be 0 because x1 is an ID
df.select(
    pds.query_cond_entropy("y", "x1")
)

y
f64
-0.0


In [15]:
# Only want singular values (principal values?)
df.select(
    pds.query_singular_values("a", "b", "x1")
)

a
list[f64]
"[288675.133152, 28.970848, 28.740724]"


In [16]:
# Singular values + The principal components
df.select(
    pds.query_pca("a", "b")
).unnest("a")

singular_value,weight_vector
f64,list[f64]
28.971787,"[-0.688449, 0.725285]"
28.74085,"[0.725285, 0.688449]"


In [17]:
# PC1
df.select(
    pds.query_principal_components("a", "b", k =1)
).unnest("principal_components").head()

pc1
f64
0.255929
0.037732
-0.234149
-0.061835
-0.414104


# ML Metrics

In [18]:
df.group_by("dummy_groups").agg(
    pds.query_l2("actual", "predicted").alias("l2"),
    pds.query_log_loss("actual", "predicted").alias("log loss"),
    pds.query_binary_metrics(actual="actual", pred="predicted").alias("combo")
).unnest("combo")


dummy_groups,l2,log loss,precision,recall,f,average_precision,roc_auc
str,f64,f64,f64,f64,f64,f64,f64
"""a""",0.330901,0.995374,0.491622,0.491622,0.491622,0.493571,0.504158
"""b""",0.331714,0.993946,0.491138,0.506911,0.4989,0.503057,0.505655


# Str Extension

In [19]:
size = 100_000
df2 = pl.DataFrame({
    "sen":["Hello, world! I'm going to church."] * size,
    "word":["words", "word"] * (size //2)
})
df2.head()

sen,word
str,str
"""Hello, world! I'm going to chu…","""words"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""words"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""words"""


In [20]:
# Tokenize
df2.select(
    pds.str_tokenize(pl.col("sen").str.to_lowercase()).explode().unique()
)

sen
str
"""to"""
"""world"""
"""church"""
"""hello"""
"""going"""


In [21]:
df2.select(
    pds.str_tokenize(pl.col("sen").str.to_lowercase(), stem=True).explode().unique()
)

sen
str
"""hello"""
"""world"""
"""church"""
"""go"""
""""""


In [22]:
df2.select(
    pds.str_leven("word", pl.lit("world"))
).head()

word
u32
2
1
2
1
2


In [23]:
# Damerau-Levenshtein
df2.select(
    pds.str_d_leven("word", pl.lit("world"))
).head()

word
u32
2
1
2
1
2


In [24]:
df2.select( # column "word" vs. the word "world"
    pds.str_leven("word", pl.lit("world"), return_sim = True)
).head()

word
f64
0.6
0.8
0.6
0.8
0.6


In [25]:
df2.filter(
    # This is way faster than computing ditance and then doing a filter
    pds.filter_by_levenshtein(pl.col("word"), pl.lit("world"), 1) # <= 1. 
).head()

sen,word
str,str
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""word"""


In [26]:
df = pl.DataFrame({
    "word":["apple", "banana", "pineapple", "asasasas", "sasasass"],
    "other_data": [1,2,3,4,5]
})
gibberish = ["asasasa", "sasaaasss", "asdasadadfa"]

In [27]:
df.filter(
    pds.similar_to_vocab(
        pl.col("word"),
        vocab = gibberish,
        threshold = 0.5,
        metric = "lv", # Levenshtein similarity. Other options: dleven, osa, jw
        strategy = "any" # True if the word is similar to any word in vocab. Other options: "all", "avg"
    )
)

word,other_data
str,i64
"""asasasas""",4
"""sasasass""",5


In [28]:
df.select(
    pds.str_leven("word", pl.lit("asasasa"), return_sim=True).alias("asasasa"),
    pds.str_leven("word", pl.lit("sasaaasss"), return_sim=True).alias("sasaaasss"),
    pds.str_leven("word", pl.lit("asdasadadfa"), return_sim=True).alias("asdasadadfa"),
    pds.str_fuzz("word", pl.lit("apples")).alias("LCS based Fuzz match - apples"),
    pds.str_osa("word", pl.lit("apples"), return_sim=True).alias("Optimal String Alignment - apples"),
    pds.str_jw("word", pl.lit("apples")).alias("Jaro-Winkler - apples"),
)


asasasa,sasaaasss,asdasadadfa,LCS based Fuzz match - apples,Optimal String Alignment - apples,Jaro-Winkler - apples
f64,f64,f64,f64,f64,f64
0.142857,0.111111,0.090909,0.833333,0.833333,0.966667
0.428571,0.333333,0.272727,0.166667,0.0,0.444444
0.111111,0.111111,0.090909,0.555556,0.444444,0.5
0.875,0.666667,0.545455,0.25,0.25,0.527778
0.75,0.777778,0.454545,0.25,0.25,0.527778


# Stats Extension

In [29]:
import numpy as np

df = pl.DataFrame({
    "a": [None, None] + list(np.random.normal(size = 998))
})
df.head()

a
f64
null
null
-0.194757
1.89367
-0.264127


In [30]:
# Genenrate random numbers, respecting null positions in reference column (pl.col("a"))
df.with_columns(
    pds.random_normal(mean = 0.5, std = 1.0).alias("random_normal"),
    pl.when(pl.col("a").is_null()).then(None).otherwise(
        pds.random_normal(mean = 0.5, std = 1.0).alias("random_normal")
    ).alias("random_normal_that_respects_null_of_a")
).head()

a,random_normal,random_normal_that_respects_null_of_a
f64,f64,f64
null,-0.537838,null
null,-0.217947,null
-0.194757,1.951534,0.770303
1.89367,-1.028474,0.298758
-0.264127,0.362827,0.100426


In [31]:
# Genenrate random string
df.with_columns(
    pds.random_str(min_size = 1, max_size = 5).alias("random_str"),
    pl.when(pl.col("a").is_null()).then(None).otherwise(
        pds.random_str(min_size = 1, max_size = 5)
    ).alias("random_str_that_respects_null_of_a")
).head()

a,random_str,random_str_that_respects_null_of_a
f64,str,str
null,"""Pi3""",null
null,"""ZCnaW""",null
-0.194757,"""2""","""2"""
1.89367,"""e""","""e"""
-0.264127,"""7mg3Z""","""7mg3Z"""


In [32]:
# Genenrate fixed size random string, while respecting column a's nulls
df.with_columns(
    pl.when(pl.col("a").is_null()).then(None).otherwise(
        pds.random_str(min_size = 5, max_size = 5)
    ).alias("random_str")
).head()

a,random_str
f64,str
null,null
null,null
-0.194757,"""IeA07"""
1.89367,"""3ZtJz"""
-0.264127,"""NoAKh"""


In [33]:
df.with_columns(
    # Sample from a normal distribution, using reference column "a" 's mean and std
    pds.random_normal(pl.col("a").mean(), pl.col("a").std()).alias("test1") 
    # Sample from uniform distribution, with low = 0 and high = "a"'s max, and respect the nulls in "a"
    , pl.when(pl.col("a").is_null()).then(None).otherwise(
        pds.random(lower = 0., upper = pl.col("a").max()).alias("test2")
    )
).with_columns(
    # Add a random pertubation to test1
    pds.perturb("test1", epsilon=0.001).alias("test1_perturbed")
).head()

a,test1,literal,test1_perturbed
f64,f64,f64,f64
null,0.767885,null,0.767595
null,1.725702,null,1.725951
-0.194757,0.984246,0.355746,0.984367
1.89367,0.381581,0.420504,0.381261
-0.264127,0.618556,1.124186,0.618069


In [34]:
# New in v0.3.5
# This way, we don't have a reference column, so we cannot respect nulls, but is more convenient to use.
df.with_columns(
    pds.random().alias("[0, 1)"),
    pds.random_normal(pl.col("a").mean(), pl.col("a").std()).alias("Normal"),
    pds.random_int(0, 10).alias("Int from [0, 10)"),
).head()

a,"[0, 1)",Normal,"Int from [0, 10)"
f64,f64,f64,i32
null,0.222824,0.123143,3
null,0.516188,-1.624236,0
-0.194757,0.275714,0.469665,7
1.89367,0.41933,-0.325981,0
-0.264127,0.555975,-2.562667,0


In [35]:
# Genenrate 2 random sample, both normally distributed
# Run Welch's t test on them, p value should be big since they have equal mean
# Run a normality test. Again, p value should be big since they are normally distributed 

df.with_columns(
    pds.random_normal(0.5, 1.0).alias("test1"),
    pds.random_normal(0.5, 2.0).alias("test2"),
).select(
    pds.query_ttest_ind("test1", "test2", equal_var=False).alias("t-test"),
    pds.normal_test("test1").alias("normality_test")
).select(
    pl.col("t-test").struct.field("statistic").alias("t-tests: statistics")
    , pl.col("t-test").struct.field("pvalue").alias("t-tests: pvalue")
    , pl.col("normality_test").struct.field("statistic").alias("normality_test: statistics")
    , pl.col("normality_test").struct.field("pvalue").alias("normality_test: pvalue")
)

t-tests: statistics,t-tests: pvalue,normality_test: statistics,normality_test: pvalue
f64,f64,f64,f64
1.784321,0.074577,0.528831,0.767655


In [36]:
size = 5_000
df = pl.DataFrame({
    "market_id": range(size),
}).with_columns(
    pl.col("market_id").mod(3),
    var1 = pds.random(),
    var2 = pds.random(),
    category_1 = pds.random_int(0, 5),
    category_2 = pds.random_int(0, 10),
)

df.head(5)

market_id,var1,var2,category_1,category_2
i64,f64,f64,i32,i32
0,0.461065,0.658996,3,9
1,0.60301,0.091558,2,6
2,0.076062,0.13925,4,1
0,0.419069,0.848895,3,8
1,0.509832,0.659113,2,2


In [37]:
# In dataframe statistical tests!
df.select(
    pds.query_ttest_ind("var1", "var2", equal_var=True).alias("t-test"),
    pds.query_chi2("category_1", "category_2").alias("chi2-test"),
    pds.query_f_test("var1", group = "category_1").alias("f-test")
)

t-test,chi2-test,f-test
struct[2],struct[2],struct[2]
"{-1.4873,0.136967}","{38.462189,0.35866}","{0.893678,0.466694}"


In [38]:
# Can also be done in group by context
df.group_by("market_id").agg(
    pds.query_ttest_ind("var1", "var2", equal_var=False).alias("t-test"),
    pds.query_chi2("category_1", "category_2").alias("chi2-test"),
    pds.query_f_test("var1", group = "category_1").alias("f-test")
)

market_id,t-test,chi2-test,f-test
i64,struct[2],struct[2],struct[2]
0,"{-0.891592,0.372676}","{33.011996,0.611472}","{0.335598,0.854098}"
1,"{0.167211,0.867214}","{31.682392,0.674095}","{0.584118,0.67418}"
2,"{-1.85503,0.06368}","{43.457489,0.183527}","{0.999737,0.406474}"


In [39]:
# Benford's law
df.select(
    first_digit_cnt = pds.query_first_digit_cnt(pl.col("var1")).explode()
).with_columns(
    # This doesn't follow benford's law because it is random data
    first_digit_distribution = pl.col("first_digit_cnt") / pl.col("first_digit_cnt").sum()
)

first_digit_cnt,first_digit_distribution
u32,f64
547,0.1094
572,0.1144
562,0.1124
562,0.1124
613,0.1226
488,0.0976
516,0.1032
581,0.1162
559,0.1118


# Nearest Neighbors Related Tasks

These queries can be very slow when data/dimension gets huge, even when processed in parallel.

In [40]:
import polars_ds as pds
size = 2000
df = pl.DataFrame({
    "id": range(size), 
}).with_columns(
    pds.random().alias("var1"),
    pds.random().alias("var2"),
    pds.random().alias("var3"),
    pds.random().alias("r"),
    (pds.random() * 10).alias("rh"),
    pl.col("id").cast(pl.UInt32)
)

In [41]:
# Get neighbor count. The point itself is always considered a neighbor to itself.
df.with_columns(
    pds.query_nb_cnt(
        0.1, # radius 
        pl.col("var1"), "var2", "var3", # Columns used as the coordinates in n-d space, str | pl.Expr 
        dist = "inf", # L Infinity distance 
        parallel = True 
    ).alias("nb_l_inf_cnt")
).head() 

id,var1,var2,var3,r,rh,nb_l_inf_cnt
u32,f64,f64,f64,f64,f64,u32
0,0.78742,0.039981,0.970969,0.535949,6.958554,8
1,0.459572,0.530897,0.815052,0.564772,0.918177,21
2,0.303721,0.98004,0.656727,0.596589,1.330739,9
3,0.156137,0.147521,0.348696,0.43929,0.050149,24
4,0.948207,0.784414,0.388157,0.963052,5.550648,10


In [42]:
df.with_columns(
    pds.query_nb_cnt(
        pl.col("r"), # radius be an expression too
        "var1", "var2", "var3", # Columns used as the coordinates in n-d space, str | pl.Expr 
        dist = "l1", # L 1 distance 
        parallel = True 
    ).alias("nb_l1_r_cnt")
).head()

id,var1,var2,var3,r,rh,nb_l1_r_cnt
u32,f64,f64,f64,f64,f64,u32
0,0.78742,0.039981,0.970969,0.535949,6.958554,129
1,0.459572,0.530897,0.815052,0.564772,0.918177,413
2,0.303721,0.98004,0.656727,0.596589,1.330739,287
3,0.156137,0.147521,0.348696,0.43929,0.050149,190
4,0.948207,0.784414,0.388157,0.963052,5.550648,818


In [43]:
# Get ids of the k nearest neighbors. 
# The point itself is always considered a neighbor to itself, so k + 1 elements will be returned.
df.with_columns(
    pds.query_knn_ptwise(
        pl.col("var1"), pl.col("var2"), pl.col("var3"), # Columns used as the coordinates in n-d space
        index = "id",  # pl.col("id"), str | pl.Expr
        k = 3, 
        dist = "l2", # squared l2
        parallel = True
    ).alias("best friends")
).head() 

id,var1,var2,var3,r,rh,best friends
u32,f64,f64,f64,f64,f64,list[u32]
0,0.78742,0.039981,0.970969,0.535949,6.958554,"[0, 453, … 114]"
1,0.459572,0.530897,0.815052,0.564772,0.918177,"[1, 463, … 615]"
2,0.303721,0.98004,0.656727,0.596589,1.330739,"[2, 784, … 1510]"
3,0.156137,0.147521,0.348696,0.43929,0.050149,"[3, 1561, … 1493]"
4,0.948207,0.784414,0.388157,0.963052,5.550648,"[4, 1485, … 1188]"


In [44]:
# Get all neighbors within radius r
# The point itself is always considered a neighbor to itself.
print(df.select(
    pl.col("id"),
    pds.query_radius_ptwise(
        pl.col("var1"), pl.col("var2"), pl.col("var3"), # Columns used as the coordinates in n-d space
        index = pl.col("id"),
        r = 0.1, 
        dist = "l2", # actually this is squared l2
        parallel = True
    ).alias("best friends"),
).with_columns( # -1 to remove the point itself
    (pl.col("best friends").list.len() - 1).alias("best friends count")
).head())

shape: (5, 3)
┌─────┬──────────────────┬────────────────────┐
│ id  ┆ best friends     ┆ best friends count │
│ --- ┆ ---              ┆ ---                │
│ u32 ┆ list[u32]        ┆ u32                │
╞═════╪══════════════════╪════════════════════╡
│ 0   ┆ [0, 453, … 95]   ┆ 90                 │
│ 1   ┆ [1, 463, … 560]  ┆ 249                │
│ 2   ┆ [2, 784, … 1146] ┆ 154                │
│ 3   ┆ [3, 1561, … 630] ┆ 221                │
│ 4   ┆ [4, 1485, … 789] ┆ 140                │
└─────┴──────────────────┴────────────────────┘


In [45]:
# Get ids of the k nearest neighbors and distances
# The point itself is always considered a neighbor to itself, so k + 1 elements will be returned.
df.with_columns(
    pds.query_knn_ptwise(
        pl.col("var1"), pl.col("var2"), pl.col("var3"), # Columns used as the coordinates in n-d space
        index = pl.col("id"),
        k = 3, 
        dist = "l2", # actually this is squared l2
        parallel = True,
        return_dist = True
    ).alias("best_friends_w_dist")
).unnest("best_friends_w_dist").head()

id,var1,var2,var3,r,rh,idx,dist
u32,f64,f64,f64,f64,f64,list[u32],list[f64]
0,0.78742,0.039981,0.970969,0.535949,6.958554,"[0, 453, … 114]","[0.0, 0.000254, … 0.001885]"
1,0.459572,0.530897,0.815052,0.564772,0.918177,"[1, 463, … 615]","[0.0, 0.004463, … 0.006928]"
2,0.303721,0.98004,0.656727,0.596589,1.330739,"[2, 784, … 1510]","[0.0, 0.00233, … 0.003642]"
3,0.156137,0.147521,0.348696,0.43929,0.050149,"[3, 1561, … 1493]","[0.0, 0.000056, … 0.003643]"
4,0.948207,0.784414,0.388157,0.963052,5.550648,"[4, 1485, … 1188]","[0.0, 0.004609, … 0.009129]"


In [46]:
# Filter to only points near the given point
df.filter(
    pds.within_dist_from(
        pl.col("var1"), pl.col("var2"), pl.col("var3"), # Columns used as the coordinates in n-d space
        pt = [0.5, 0.5, 0.5],
        r = 0.2,
        dist = "l2" # actually this is squared l2, so this is asking for squared l2 <= 0.2
    )
).head()

id,var1,var2,var3,r,rh
u32,f64,f64,f64,f64,f64
1,0.459572,0.530897,0.815052,0.564772,0.918177
10,0.177432,0.463327,0.354092,0.932493,9.520162
12,0.193013,0.267148,0.308096,0.441098,1.657274
14,0.33991,0.684942,0.831401,0.979315,1.569871
21,0.486707,0.629695,0.392604,0.83484,6.098153


In [47]:
# Haversine distance is available when dimension is 2
df.filter(
    pds.within_dist_from(
        pl.col("var1"), pl.col("var2"), # Columns used as the coordinates in n-d space
        pt = [0.5, 0.5],
        r = 10, # in km
        dist = "h" 
    )
).head()

id,var1,var2,var3,r,rh
u32,f64,f64,f64,f64,f64
1,0.459572,0.530897,0.815052,0.564772,0.918177
36,0.52125,0.487166,0.165547,0.853934,5.989968
43,0.548018,0.4681,0.667315,0.235272,7.258679
96,0.468322,0.422567,0.026001,0.349229,0.946792
157,0.528324,0.414781,0.435373,0.74664,7.015706


In [48]:
df.filter(
    pds.within_dist_from(
        pl.col("var1"), pl.col("var2"), 
        pt = [0.5, 0.5],
        # radius can also be an existing column in the dataframe.
        r = pl.col("rh"), 
        dist = "h" 
    )
).head()

id,var1,var2,var3,r,rh
u32,f64,f64,f64,f64,f64
36,0.52125,0.487166,0.165547,0.853934,5.989968
43,0.548018,0.4681,0.667315,0.235272,7.258679
399,0.459904,0.529395,0.477996,0.21315,9.990249
615,0.497956,0.55592,0.884539,0.697242,7.829531
673,0.515309,0.506294,0.7482,0.989028,9.553494


In [49]:
friends = df.select(
    pl.col("id").cast(pl.UInt64),
    pds.query_radius_ptwise(
        # Columns used as the coordinates in n-d space
        pl.col("var1"), pl.col("var2"), 
        index=pl.col("id"),
        r = 0.02, 
        dist = "l2",
    ).alias("friends")
).with_columns(
    pl.col("friends").list.len().alias("count")
)
friends.head()

id,friends,count
u64,list[u32],u32
0,"[0, 1197, … 1890]",83
1,"[1, 399, … 1579]",133
2,"[2, 1907, … 362]",77
3,"[3, 1561, … 770]",136
4,"[4, 1560, … 901]",84


# String Nearest Neighbors

This might be very slow for very large vocab / column.

In [50]:
df = pl.DataFrame({
    "a":["AAAAA", "ABCABC", "AAAADDD", "ADSDSDS", "WORD"],
    "b":["AAAAT", "ABCACD", "ADSSD", "APPLES", "WORLD"] 
})

In [51]:
# Use Levenshtein to find the nearest neighbor in vocab to word in column a
df.select(
    pds.query_similar_words(
        "a",
        vocab = pl.col("b"),
        k = 1, 
        metric = "lv"
    ).alias("similar_words_from_vocab"),
)

similar_words_from_vocab
str
"""AAAAT"""
"""ABCACD"""
"""AAAAT"""
"""ADSSD"""
"""WORLD"""


In [52]:
# Use Levenshtein to find 2 nearest neighbors
df.select(
    pds.query_similar_words(
        "a",
        vocab = pl.col("b"),
        k = 2, 
        metric = "lv"
    ).alias("similar_words_from_vocab"),
)

similar_words_from_vocab
list[str]
"[""AAAAT"", ""ADSSD""]"
"[""ABCACD"", ""AAAAT""]"
"[""AAAAT"", ""ABCACD""]"
"[""ADSSD"", ""APPLES""]"
"[""WORLD"", ""ADSSD""]"


In [53]:
# Currently only Levenshtein and hamming are implemented for this
# Empty means nothing in vocab can be compared in the hamming sense with the corresponding word in a
df.select(
    pds.query_similar_words(
        "a",
        vocab = pl.col("b"),
        k = 2, 
        threshold = 4,
        metric = "hamming"
    ).alias("similar_words_from_vocab"),
)

similar_words_from_vocab
list[str]
"[""AAAAT"", ""ADSSD""]"
"[""ABCACD""]"
[]
[]
[]


In [54]:
# You may provide a vocab like this
df.select(
    pl.col("a"),
    pds.query_similar_words(
        "a",
        vocab = ["WORLD", "AAAAA", "ABCDEFG", "ZIV", "TQQQ"],
        k = 3, 
        metric = "lv"
    ).alias("similar_words_from_vocab"),
)

a,similar_words_from_vocab
str,list[str]
"""AAAAA""","[""AAAAA"", ""ZIV"", ""WORLD""]"
"""ABCABC""","[""ABCDEFG"", ""AAAAA"", ""ZIV""]"
"""AAAADDD""","[""AAAAA"", ""WORLD"", ""ABCDEFG""]"
"""ADSDSDS""","[""ABCDEFG"", ""WORLD"", ""AAAAA""]"
"""WORD""","[""WORLD"", ""ZIV"", ""TQQQ""]"


# Using PDS Expressions On Series / NumPy arrays

In [55]:
df = pds.random_data(size=100_000, n_cols=0).select(
    pds.random(0.0, 1.0).round().alias("actual"),
    pds.random(0.0, 1.0).alias("predicted"),
    pds.random_int(0, 3).alias("0-2"),
    pds.random_int(0, 10).alias("0-9"),
)
df.head()

actual,predicted,0-2,0-9
f64,f64,i32,i32
0.0,0.053652,2,1
1.0,0.858039,1,0
1.0,0.503168,1,7
0.0,0.732525,2,0
1.0,0.832277,2,6


In [56]:
pds.eval_series(
    df["0-2"], df["0-9"], # use series as args
    expr = "query_jaccard_col" # name of the pds expression
)

jaccard_col
f64
0.3


In [57]:
pds.eval_series(
    df["actual"], df["predicted"], # use series as args
    expr = "query_binary_metrics" # name of the pds expression
).unnest("binary_metrics")

precision,recall,f,average_precision,roc_auc
f64,f64,f64,f64,f64
0.501603,0.500901,0.501252,0.500733,0.501364


In [58]:
pds.eval_series(
    np.random.random(size = 1000), np.random.random(size = 1000), # can also use NumPy
    expr = "query_psi", # name of the pds expression
    n_bins = 5, 
)

psi
f64
0.008491
